# Optimize the Model

In [1]:
# Create a new Jupyter Notebook file and name it AlphabetSoupCharity_Optimzation.ipynb.

In [2]:
# 2.Import your dependencies, and read in the charity_data.csv to a Pandas DataFrame.
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# 3. Preprocess the dataset like you did in Step 1, taking into account any modifications to optimize the model.

In [4]:
# # Adjusting the input data to ensure that there are no variables or outliers that are causing confusion in the model, such as:
# # Dropping more or fewer columns.
# # Creating more bins for rare occurrences in columns.
# # Increasing or decreasing the number of values for each bin.
# # Adding more neurons to a hidden layer.
# # Adding more hidden layers.
# # Using different activation functions for the hidden layers.
# Adding or reducing the number of epochs to the training regimen.

In [5]:
# Drop columns
application_df = application_df.drop(["EIN","NAME"],axis='columns')
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
ap_counts = application_df['APPLICATION_TYPE'].value_counts()
ap_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(ap_counts[ap_counts < 600].index)


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1245        1
C5200        1
C2190        1
C2150        1
C1820        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# For those columns that have more than 10 unique values, determine the number of data points for each unique value.
class_counts[class_counts > 10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 500].index)


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
ap_dummies_df = pd.get_dummies(application_df)

In [13]:
# Split our preprocessed data into our features and target arrays
X = ap_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = ap_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
no_input_features = len(X_train[0])
hidden_nodes_1 = 99
hidden_nodes_2 = 50
hidden_nodes_3 = 20
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=no_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_2, input_dim=no_input_features, activation="sigmoid")
)

# Add- third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_3, input_dim=no_input_features, activation="sigmoid")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 99)                4356      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5000      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 10,397
Trainable params: 10,397
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 4.Design a neural network model, taking into account any modifications that will optimize the model to achieve higher than 75% accuracy.

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 843us/step - loss: 0.5958 - accuracy: 0.6981
Epoch 2/100
804/804 [==============================] - 1s 816us/step - loss: 0.5720 - accuracy: 0.7230
Epoch 3/100
804/804 [==============================] - 1s 825us/step - loss: 0.5656 - accuracy: 0.7247
Epoch 4/100
804/804 [==============================] - 1s 804us/step - loss: 0.5606 - accuracy: 0.7265
Epoch 5/100
804/804 [==============================] - 1s 811us/step - loss: 0.5577 - accuracy: 0.7278
Epoch 6/100
804/804 [==============================] - 1s 808us/step - loss: 0.5561 - accuracy: 0.7284
Epoch 7/100
804/804 [==============================] - 1s 816us/step - loss: 0.5547 - accuracy: 0.7284
Epoch 8/100
804/804 [==============================] - 1s 833us/step - loss: 0.5534 - accuracy: 0.7286
Epoch 9/100
804/804 [==============================] - 1s 818us/step - loss: 0.5526 - accuracy: 0.7299
Epoch 10/100
804/804 [==============================] - 1s 822us/step - l

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# 5.Save and export your results to an HDF5 file, and name it AlphabetSoupCharity_Optimization.h5.

In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5.")

# Step 4: Write a Report on the Neural Network Model